# Caption

## Performance-test #1: using Arrays of Arrays vs 3 dimensional Arrays

### two little examples

In [2]:
ArrayA= [[1.0 0.0;0.0 1.0],[1.0 2.0;3.0 4.0]]

2-element Array{Array{Float64,2},1}:
 [1.0 0.0; 0.0 1.0]
 [1.0 2.0; 3.0 4.0]

It is now a vector and its entries are matrices

In [51]:
ArrayA[1]

2×2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0

In [2]:
Array3d= Array{Float64,3}(undef,2,2,2)
Array3d[1,:,:]=[1.0 0.0;0.0 1.0]

2×2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0

In [3]:
Array3d[2,:,:]=[1.0 2.0;3.0 4.0]

2×2 Array{Float64,2}:
 1.0  2.0
 3.0  4.0

basically the same Array:

In [4]:
(Array3d[1,:,:]==ArrayA[1][:,:]) && (Array3d[2,:,:]==ArrayA[2][:,:])

true

### defining two test-functions

actual code iterates over every index and does a simple operation, so the test-functions should do the same

#### test Arrays of Arrays

In [5]:
function arrayarray(A::Array{Array{Float64,2},1}, B::Array{Array{Float64,2},1})
    sum=0.0;
    for k in 1:size(A[1])[2]
        for l in 1:size(A[1])[1]
            for j=1:length(B)
                for i=1:length(A)
                    sum+=A[i][l,k]*B[j][l,k]
                end
            end
        end
    end
 return sum;
end

arrayarray (generic function with 1 method)

#### test 3 dimensional Arrays

In [50]:
function array3d(A::Array{Float64,3}, B::Array{Float64,3})
    sum=0.0;
    for k in 1:size(A)[3]
        for l in 1:size(A)[2]
            for i=1:size(A)[1]
                for j=1:size(B)[1]
                    sum+=A[i,l,k]*B[j,l,k]
                end
            end
        end
    end
 return sum;
end

array3d (generic function with 1 method)

### initialize large, equivalent Matrices to test

first one has random entries, second one is filled with the exact same entries

In [7]:
A1=Array{Array{Float64,2},1}(undef,10)
B1=Array{Array{Float64,2},1}(undef,10)

A2=Array{Float64,3}(undef,10,1000,1000)
B2=Array{Float64,3}(undef,10,1000,1000)

for i in 1:10
    A1[i]=rand(1000,1000)
    B1[i]=rand(1000,1000)

    A2[i,:,:]=A1[i]
    B2[i,:,:]=B1[i]
end

### do the test

In [8]:
using BenchmarkTools

In [9]:
sum1=0.0; sum2=0.0;

In [10]:
benchmarksarrayarray = @benchmark sum1=arrayarray(A1,B1);

In [11]:
benchmarksarray3d = @benchmark sum2=array3d(A2,B2);

proving that both functions have the same result:

In [12]:
sum1==sum2

true

"old" version:

In [13]:
benchmarksarrayarray

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     227.629 ms (0.00% GC)
  median time:      230.838 ms (0.00% GC)
  mean time:        230.995 ms (0.00% GC)
  maximum time:     235.293 ms (0.00% GC)
  --------------
  samples:          22
  evals/sample:     1

"new" version:

In [14]:
benchmarksarray3d

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     87.117 ms (0.00% GC)
  median time:      89.741 ms (0.00% GC)
  mean time:        89.831 ms (0.00% GC)
  maximum time:     92.374 ms (0.00% GC)
  --------------
  samples:          56
  evals/sample:     1

## Performance-test #2: Comparing slices to views and how to use them properly

### defining a function to test slices

In [26]:
function testslice(vector,a)
    for i in 1:1000000
        a=vector[i:i+5];
    end
end

testslice (generic function with 1 method)

### defining a function to test for-loops instead of slices

In [27]:
function testloop(vector,a)
    for i in 1:1000000
        for j in 0:5
            a[j+1]=vector[i+j]
        end
    end
end

testloop (generic function with 1 method)

### defining a function to test @view

In [28]:
function testview(vector)
    for i in 1:1000000
        a= @view vector[i:i+5];
    end
end

testview (generic function with 1 method)

### defining a function to test preallocating the @view

In [29]:
function testpreallocview(vector,a)
    for i in 1:1000000
        a= @view vector[i:i+5];
    end
end

testpreallocview (generic function with 1 method)

### defining a function to test @view in combination with @inbounds

In [30]:
function testviewinbounds(vector)
    for i in 1:1000000
        a= @inbounds @views vector[i:i+5];
    end
end

testviewinbounds (generic function with 1 method)

### initializing a testvector and preallocating a slice/@view

In [31]:
vector = rand(1000000+5);

In [32]:
as = vector[1:6];

In [33]:
av = @view vector[1:6];

### benchmarking every function

In [34]:
@benchmark testslice(vector,as)

BenchmarkTools.Trial: 
  memory estimate:  122.07 MiB
  allocs estimate:  1000000
  --------------
  minimum time:     36.204 ms (7.26% GC)
  median time:      37.153 ms (8.55% GC)
  mean time:        37.231 ms (8.25% GC)
  maximum time:     40.787 ms (8.86% GC)
  --------------
  samples:          135
  evals/sample:     1

In [35]:
@benchmark testloop(vector,as)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     5.429 ms (0.00% GC)
  median time:      5.579 ms (0.00% GC)
  mean time:        5.646 ms (0.00% GC)
  maximum time:     7.085 ms (0.00% GC)
  --------------
  samples:          884
  evals/sample:     1

In [36]:
@benchmark testview(vector)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     533.139 μs (0.00% GC)
  median time:      533.255 μs (0.00% GC)
  mean time:        539.323 μs (0.00% GC)
  maximum time:     857.694 μs (0.00% GC)
  --------------
  samples:          9257
  evals/sample:     1

In [37]:
@benchmark testpreallocview(vector,av)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     533.154 μs (0.00% GC)
  median time:      533.256 μs (0.00% GC)
  mean time:        539.131 μs (0.00% GC)
  maximum time:     860.568 μs (0.00% GC)
  --------------
  samples:          9260
  evals/sample:     1

In [38]:
@benchmark testviewinbounds(vector)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     14.391 ns (0.00% GC)
  median time:      14.438 ns (0.00% GC)
  mean time:        14.713 ns (0.00% GC)
  maximum time:     42.424 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     998

BUT @inbounds is dangerous:

In [46]:
vector=rand(10);

In [47]:
testview(vector);

BoundsError: BoundsError: attempt to access 10-element Array{Float64,1} at index [6:11]

rewrite testviewinbounds to see what's actually happening:

In [48]:
function testviewinbounds(vector)
    a=@views vector[1:6];
    for i in 1:1000000
        a= @inbounds @views vector[i:i+5];
    end
    println(a);
end

testviewinbounds (generic function with 1 method)

In [49]:
testviewinbounds(vector);

[2.28095e-314, 9.88131e-324, 2.21233e-314, 2.28865e-314, 2.28096e-314, 5.92879e-323]
